In [ ]:
!git clone https://github.com/hsma-programme/Teaching_DES_Concepts_Streamlit.git
!pip install simpy

import gc
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
%cd /content/Teaching_DES_Concepts_Streamlit
from model_classes import Scenario, multiple_replications
from output_animation_functions import reshape_for_animations, animate_activity_log

In [ ]:
n_triage = 4
prob_trauma = 0.3
n_trauma = 6
n_cubicles_2 = 6
n_reg = 3
n_exam = 3
n_cubicles_1 = 2 
non_trauma_treat_p = 0.7 

seed = 42
n_reps = 3
run_time_days = 5

# Animated Log

In [ ]:
args = Scenario(
        random_number_set=seed,
                 n_triage=n_triage,
                 n_reg=n_reg,
                 n_exam=n_exam,
                 n_trauma=n_trauma,
                 n_cubicles_1=n_cubicles_1,
                 n_cubicles_2=n_cubicles_2,
                 non_trauma_treat_p=non_trauma_treat_p,
                 prob_trauma=prob_trauma)

detailed_outputs = multiple_replications(
                args,
                n_reps=n_reps,
                rc_period=run_time_days*60*24,
                return_detailed_logs=True
            )


results = pd.concat([detailed_outputs[i]['results']['summary_df'].assign(rep= i+1)
                            for i in range(n_reps)]).set_index('rep')

full_event_log = pd.concat([detailed_outputs[i]['results']['full_event_log'].assign(rep= i+1)
                            for i in range(n_reps)])

del detailed_outputs
gc.collect()

animation_dfs_log = reshape_for_animations(
    full_event_log=full_event_log[
        (full_event_log['rep']==1) &
        ((full_event_log['event_type']=='queue') | (full_event_log['event_type']=='resource_use')  | (full_event_log['event_type']=='arrival_departure')) &
            # Limit to first 5 days
        (full_event_log['time'] <= 60*24*5)
    ],
    every_x_minutes=5
)['full_patient_df']

del full_event_log
gc.collect()

event_position_df = pd.DataFrame([
                # {'event': 'arrival', 'x':  10, 'y': 250, 'label': "Arrival" },
                
                # Triage - minor and trauma                
                {'event': 'triage_wait_begins', 
                 'x':  160, 'y': 400, 'label': "Waiting for<br>Triage"  },
                {'event': 'triage_begins', 
                 'x':  160, 'y': 315, 'resource':'n_triage', 'label': "Being Triaged" },
            
                # Minors (non-trauma) pathway 
                {'event': 'MINORS_registration_wait_begins', 
                 'x':  300, 'y': 145, 'label': "Waiting for<br>Registration"  },
                {'event': 'MINORS_registration_begins', 
                 'x':  300, 'y': 85, 'resource':'n_reg', 'label':'Being<br>Registered'  },

                {'event': 'MINORS_examination_wait_begins', 
                 'x':  465, 'y': 145, 'label': "Waiting for<br>Examination"  },
                {'event': 'MINORS_examination_begins', 
                 'x':  465, 'y': 85, 'resource':'n_exam', 'label': "Being<br>Examined" },

                {'event': 'MINORS_treatment_wait_begins', 
                 'x':  630, 'y': 145, 'label': "Waiting for<br>Treatment"  },
                {'event': 'MINORS_treatment_begins', 
                 'x':  630, 'y': 85, 'resource':'n_cubicles_1', 'label': "Being<br>Treated" },

                # Trauma pathway
                {'event': 'TRAUMA_stabilisation_wait_begins', 
                 'x': 300, 'y': 560, 'label': "Waiting for<br>Stabilisation" },
                {'event': 'TRAUMA_stabilisation_begins', 
                 'x': 300, 'y': 500, 'resource':'n_trauma', 'label': "Being<br>Stabilised" },

                {'event': 'TRAUMA_treatment_wait_begins', 
                 'x': 630, 'y': 560, 'label': "Waiting for<br>Treatment" },
                {'event': 'TRAUMA_treatment_begins', 
                 'x': 630, 'y': 500, 'resource':'n_cubicles_2', 'label': "Being<br>Treated" },

                 {'event': 'exit', 
                 'x':  670, 'y': 330, 'label': "Exit"}
            ])

animated_plot = animate_activity_log(
                    full_patient_df=animation_dfs_log[animation_dfs_log["minute"]<=60*24*5],
                    event_position_df = event_position_df,
                    scenario=args,
                    include_play_button=True,
                    return_df_only=False,
                    plotly_height=900,
                    plotly_width=1600,
                    override_x_max=700,
                    override_y_max=675,
                    icon_and_text_size=24,
                    display_stage_labels=False,
                    wrap_queues_at=10,
                    time_display_units="dhm",
                    # show_animated_clock=True,
                    # animated_clock_coordinates = [100, 50],
                    add_background_image="https://raw.githubusercontent.com/hsma-programme/Teaching_DES_Concepts_Streamlit/main/resources/Full%20Model%20Background%20Image%20-%20Horizontal%20Layout.drawio.png",
            )

del animation_dfs_log
gc.collect()

animated_plot 

# Simple Metrics

In [ ]:
in_range_util = sum((results.mean().filter(like="util")<0.85) & (results.mean().filter(like="util") > 0.65))
in_range_wait = sum((results.mean().filter(like="wait")<120))
                    

print(":bed: **Utilisation Metrics in Ideal Range**", value="{} of {}".format(in_range_util, len(results.mean().filter(like="util"))))

#util_fig_simple = px.bar(results.mean().filter(like="util"), opacity=0.5)
print(
"""
The emergency department wants to aim for an average of 65% to 85% utilisation across all resources in the emergency department. 

The green box shows this ideal range. If the bars overlap with the green box, utilisation is ideal. 

If utilisation is below this, you might want to **reduce** the number of those resources available. 

If utilisation is above this point, you may want to **increase** the number of that type of resource available.
"""
)
util_fig_simple = go.Figure()
# Add optimum range
util_fig_simple.add_hrect(y0=0.65, y1=0.85,
                         fillcolor="#5DFDA0", opacity=0.25,  line_width=0)
# Add extreme range (above)
util_fig_simple.add_hrect(y0=0.85, y1=1,
                         fillcolor="#D45E5E", opacity=0.25, line_width=0)
# Add suboptimum range (below)
util_fig_simple.add_hrect(y0=0.4, y1=0.65,
                         fillcolor="#FDD049", opacity=0.25, line_width=0)
# Add extreme range (below)
util_fig_simple.add_hrect(y0=0, y1=0.4,
                         fillcolor="#D45E5E", opacity=0.25, line_width=0)

util_fig_simple.add_bar(x=results.mean().filter(like="util").index.tolist(),
                    y=results.mean().filter(like="util").tolist())

util_fig_simple.update_layout(yaxis_tickformat = '.0%')
util_fig_simple.update_yaxes(title_text='Resource Utilisation (%)',
                         range=[-0.05, 1.1])
# util_fig_simple.data = util_fig_simple.data[::-1]
util_fig_simple.update_xaxes(labelalias={
"01b_triage_util": "Triage<br>Bays", 
"02b_registration_util": "Registration<br>Cubicles",
"03b_examination_util": "Examination<br>Bays",
"04b_treatment_util(non_trauma)": "Treatment<br>Bays<br>(non-trauma)",
"06b_trauma_util": "Stabilisation<br>Bays",
"07b_treatment_util(trauma)": "Treatment<br>Bays<br>(trauma)"
}, tickangle=0)
                
util_fig_simple